In [1]:
import datajoint as dj
dj.conn()
import csv

Connecting root@localhost:3306


In [2]:
from workflow_session.pipeline import lab, subject, session
from workflow_session.ingest import *

In [3]:
ingest_lab(); ingest_subjects();ingest_sessions()


---- Insert 2 entry(s) into lab tables ----

---- Insert 2 entry(s) into project table ----

---- Insert entry(s) into publication/keyword tables ----

---- Insert 2 entry(s) into protocol tables ----

---- Insert 2 entry(s) into subject tables ----

---- Insert 2 entry(s) into session.Session ----


In [4]:
print(lab.Lab & 'lab="LabA"')
print(lab.Project & 'project="ProjA"')
print(lab.Protocol() & 'protocol="ProtA"')

lab_info = (lab.Lab & 'lab="LabA"').fetch1()
proj_info = (lab.Project & 'project="ProjA"').fetch1()
prot_info = (lab.Protocol() & 'protocol="ProtA"').fetch1()

*lab     lab_name       institution    address        time_zone    
+------+ +------------+ +------------+ +------------+ +-----------+
LabA     The Example La Example Uni    221B Baker St, UTC+0        
 (Total: 1)

*project    project_descri repositoryurl  repositoryname pharmacology   viruses     slices     stimulus       surgery    
+---------+ +------------+ +------------+ +------------+ +------------+ +---------+ +--------+ +------------+ +---------+
ProjA       Example projec https://github element-lab                                          videos generat            
 (Total: 1)

*protocol    protocol_type  protocol_descr
+----------+ +------------+ +------------+
ProtA        IRB expedited  Protocol for m
 (Total: 1)



In [5]:
import pynwb
from element_session.export import *
session_key=(session.Session&'subject="subject5"').fetch1('KEY')
mylab_key = (lab.Lab & 'lab="LabA"').fetch1('KEY')
myproj_key= (lab.Project & 'project="ProjA"').fetch1('KEY')
myprot_key= (lab.Protocol() & 'protocol="ProtA"').fetch1('KEY')
session_to_nwb(session_key,lab_key=mylab_key,project_key=myproj_key)

/Users/cb/miniconda3/envs/venv-nwb/lib/python3.8/site-packages/pynwb/file.py:753: UserWarning: Date is missing timezone information. Updating to local timezone.
  warn("Date is missing timezone information. Updating to local timezone.")


root pynwb.file.NWBFile at 0x140690194467040
Fields:
  file_create_date: [datetime.datetime(2021, 12, 1, 14, 22, 4, 992693, tzinfo=tzlocal())]
  identifier: subject5_20200415_111638
  institution: Example Uni
  session_description: Successful data collection, no notes
  session_start_time: 2020-04-15 11:16:38-05:00
  timestamps_reference_time: 2020-04-15 11:16:38-05:00

In [32]:
mynwbfile

root pynwb.file.NWBFile at 0x140337549119696
Fields:
  file_create_date: [datetime.datetime(2021, 12, 1, 14, 21, 26, 638467, tzinfo=tzlocal())]
  identifier: subject5_20200415_111638
  institution: Example Uni
  protocol: ProtA
  session_description: Successful data collection, no notes
  session_start_time: 2020-04-15 11:16:38-05:00
  timestamps_reference_time: 2020-04-15 11:16:38-05:00

In [100]:
session_key=(session.Session&'subject="subject5"').fetch1('KEY')
session_identifier = {}
for k, v in session_key.items():
    session_identifier[k] = v.strftime('%Y%m%d_%H%M%S') if isinstance(v, datetime) else v

session_info = (session.Session & session_key).join(session.SessionNote, left=True).fetch1()

def mytuple():
    identifier='_'.join(session_identifier.values()),
    session_description=session_info['session_note'] if session_info['session_note'] else '',
    session_start_time=session_info['session_datetime']
    return identifier,session_description,session_start_time
info=dict(identifier='_'.join(session_identifier.values()),
    session_description='Note',
    session_start_time=session_info['session_datetime'],
    institution='')
info={k: v for k, v in info.items() if v} #drop empty
asstring = ','.join('='.join((str(key),val)) for (key,val) in info.items())

In [12]:
a=None;b=None;c=None
if [x for x in (a, b,c) if x is not None]: print('g')